In [31]:
import pandas as pd
import numpy as np
import csv
import json
from datetime import datetime
import matplotlib.pyplot as plt
import xgboost as xgb

In [32]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [33]:
train.head(10)

,T_battery_power,T_blue,T_clock_speed,T_dual_sim,T_fc,T_four_g,T_int_memory,T_m_dep,T_mobile_wt,T_n_cores,...,T_px_height,T_px_width,T_ram,T_sc_h,T_sc_w,T_talk_time,T_three_g,T_touch_screen,T_wifi,price_range
0,842,NaN,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1.0,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1.0,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1.0,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1.0,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0.0,0.5,1,3,0,22,0.7,164,1,...,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0.0,1.7,0,4,1,10,0.8,139,8,...,381,1018,3220,13,8,18,1,0,1,3
7,1954,0.0,0.5,1,0,0,24,0.8,187,4,...,512,1149,700,16,3,5,1,1,1,0
8,1445,1.0,0.5,0,0,0,53,0.7,174,7,...,386,836,1099,17,1,20,1,0,0,0
9,509,1.0,0.6,1,2,1,9,0.1,93,5,...,1137,1224,513,19,10,12,1,0,0,0


In [34]:
train.shape

(2000, 21)

In [35]:
train.price_range.value_counts()

3    500
2    500
1    500
0    500
Name: price_range, dtype: int64

In [36]:
test.shape

(1000, 21)

In [37]:
labels = train.price_range
test_ind = test.T_id

In [38]:
train.drop(['price_range'], axis=1, inplace=True)

In [39]:
test.drop(['T_id'], axis=1, inplace=True)

In [40]:
test.head()

,T_battery_power,T_blue,T_clock_speed,T_dual_sim,T_fc,T_four_g,T_int_memory,T_m_dep,T_mobile_wt,T_n_cores,T_pc,T_px_height,T_px_width,T_ram,T_sc_h,T_sc_w,T_talk_time,T_three_g,T_touch_screen,T_wifi
0,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
1,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
2,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
3,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
4,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1


In [41]:
train.head()

,T_battery_power,T_blue,T_clock_speed,T_dual_sim,T_fc,T_four_g,T_int_memory,T_m_dep,T_mobile_wt,T_n_cores,T_pc,T_px_height,T_px_width,T_ram,T_sc_h,T_sc_w,T_talk_time,T_three_g,T_touch_screen,T_wifi
0,842,NaN,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1.0,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1.0,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1.0,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1.0,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [12]:
from sklearn.feature_extraction import DictVectorizer

train = train.T.to_dict().values()
test = test.T.to_dict().values()


vec = DictVectorizer()
train = vec.fit_transform(train)
test = vec.transform(test)

In [27]:
param = {}
param["objective"] = "reg:linear"
param["eta"] = 0.005
param["min_child_weight"] = 100
param["max_depth"] = 6
param["subsample"] = 0.7
param["colsample_bytree"] = 0.7
param["scale_pos_weight"] = 1


offset = 5000

num_rounds = 10000
xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:, :], label = labels[offset:])
xgval = xgb.DMatrix(train[:offset, :], label = labels[:offset])

evallist = [(xgtrain, 'train'), (xgval, 'val')]

#model = xgb.train(param, xgtrain, evallist,early_stopping_rounds=120)


#preds = model.predict(xgtest, ntree_limit=model.best_iteration)

#preds = pd.DataFrame({"Id": test_ind, "price_range": preds})
#preds = preds.set_index('Id')
#preds.to_csv('xgboost.csv')

In [26]:
#clf = xgb.train(param, xgtrain, 20)

In [ ]:
model.save_model('0001.model')
model1.dump_model('dump.raw.txt')

In [22]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [23]:
y_pred = gnb.fit(train, labels).predict(test)

In [42]:
train.fillna(0, inplace=True)

In [43]:
scores = cross_val_score(gnb, train, labels, cv=5)
scores.mean() 

0.8089999999999999

In [44]:
#RF
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [117]:
clf = RandomForestClassifier(n_estimators=100,min_samples_split=4, random_state=0)
scores = cross_val_score(clf, train, labels, cv=5)
scores.mean() 

0.8795

In [46]:
#ADABOOST
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf2 = AdaBoostClassifier(n_estimators=1000)
scores = cross_val_score(clf2, train, labels, cv=5)
scores.mean()                             


0.6435

In [103]:
clf3 = GradientBoostingClassifier(n_estimators=400, learning_rate=1.0,
    max_depth=1, min_samples_leaf=1 ,min_samples_split=0.5).fit(train, labels)
x = clf3.predict(test)
scores = cross_val_score(clf3, train, labels, cv=5)
scores.mean() 

0.9275

In [64]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.025,
    max_depth=4, random_state=0, loss='ls').fit(train, labels)
y = est.predict(test)
scores = cross_val_score(est, train, labels, cv=5)
scores.mean()


In [96]:
df = pd.DataFrame(x)

In [97]:
df.to_csv('sub2.csv')

In [30]:
## Saumya's submission

import numpy as np
import xgboost as xgb
import pandas as pd

# print("reading the train and test data\n")
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

featureNames = train.columns[:-1]
# testfeatureNames = test.columns[1:]
# print("Test values\n", test[featureNames].values, "\n")
# print("Test \n", test, "\n")

# print(featureNames)
xg_train = xgb.DMatrix(train[featureNames].values, label=train['price_range'].values)
xg_test = xgb.DMatrix(test[featureNames].values, label=test['price_range'].values)

# print(max(train['price_range'].values))
# print(min(train['price_range'].values))

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['num_class'] = 4 # number of classes
param['max_depth'] = 6
param['eta'] = 0.1
param['silent'] = 1
param['nthread'] = 4

# watchlist = [(xg_train, 'train'), (xg_test, 'test')] not sure about this...
num_round = 5
bst = xgb.train(param, xg_train, num_round)

# get prediction
pred = bst.predict(xg_test)

# TODO: convert to required csv format
print(pred)

count = 0
for x in range(len(pred)):
    if pred[x] == test['price_range'].values[x]:
        count = count + 1

print((count / len(pred)) * 100, "%")


KeyError: 'price_range'